In [ ]:
%matplotlib inline
import os
#os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ[ 'NUMBA_CACHE_DIR' ] = '/tmp/'
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import scrublet as scr
import scipy.io


In [ ]:
scanpy_species= 'human'
expected_doublet_rate=0.06
scanpy_min_genes = 300
scanpy_min_cells = 5
scanpy_pct_mt = 20
scanpy_total_counts = 200

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

#### Loading data

In [ ]:
filelist_all = os.listdir("./")

filelist = [x for x in filelist_all if x.endswith("h5")]

In [ ]:
adatas = [sc.read_10x_h5(filename) for filename in filelist]
#adatas = [sc.read_10x_mtx(filename,cache=True) for filename in filelist]

In [ ]:
adatas_raw = [sc.read_10x_h5(filename) for filename in filelist]
#adatas_raw = [sc.read_10x_mtx(filename,cache=True) for filename in filelist]

#### Preprocessing

In [ ]:
for i in range(len(adatas_raw)):
  adatas_raw[i].var_names_make_unique()
  adatas_raw[i].obs['sample'] = os.path.splitext(os.path.basename(filelist[i]))[0]
  adatas_raw[i].obs['species'] = scanpy_species
  sc.pp.filter_cells(adatas_raw[i], min_genes=0)
  

In [ ]:
for i in range(len(adatas)):
  adatas[i].var_names_make_unique()
  adatas[i].obs['sample'] = os.path.splitext(os.path.basename(filelist[i]))[0]
  adatas[i].obs['species'] = scanpy_species
  sc.pp.filter_cells(adatas[i], min_genes=scanpy_min_genes)
  sc.pp.filter_genes(adatas[i], min_cells=scanpy_min_cells)

In [ ]:
for i in range(len(adatas)):
  adatas[i].var['mt'] = adatas[i].var_names.str.startswith('MT-')
  sc.pp.calculate_qc_metrics(adatas[i], qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
  adatas[i] = adatas[i][adatas[i].obs.pct_counts_mt < scanpy_pct_mt, :]
  adatas[i] = adatas[i][adatas[i].obs.total_counts > scanpy_total_counts, :] 

for i in range(len(adatas_raw)):
  adatas_raw[i].var['mt'] = adatas_raw[i].var_names.str.startswith('MT-')
  sc.pp.calculate_qc_metrics(adatas_raw[i], qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

#### Remove doublet

In [ ]:
for i in range(len(adatas)):
    sc.pp.scrublet(adatas[i],expected_doublet_rate =expected_doublet_rate)
    adatas[i] = adatas[i][adatas[i].obs['predicted_doublet']==False]

In [ ]:
adatas

In [ ]:
adatas_raw

In [ ]:
adatas_concat = ad.concat([adatas[i] for i in range(len(adatas))], merge="same")
adatas_concatraw = ad.concat([adatas_raw[i] for i in range(len(adatas_raw))], merge="same")

In [ ]:
adatas_concat

In [ ]:
adatas_concatraw

#### Cell counts bar plot for each sample before and after filter

In [ ]:
s = adatas_concatraw.obs["sample"].value_counts()
s = s.to_frame().reset_index()
ax=sns.barplot(s, x = 'sample',y = 'count',color='blue') #before filter
ax.tick_params(axis='x', labelrotation=45)

In [ ]:
s = adatas_concat.obs["sample"].value_counts()
s = s.to_frame().reset_index()
ax = sns.barplot(s, x = 'sample',y = 'count',color='orange') #after filter
ax.tick_params(axis='x', labelrotation=45)

#### Number of read counts per cell vs percentage mt genes before and after filter

In [ ]:
ax = sc.pl.scatter(adatas_concatraw, x="total_counts", y="pct_counts_mt", color="sample",show=False)
ax.axhline(y=scanpy_pct_mt,color = "r")

In [ ]:
sc.pl.scatter(adatas_concat, x="total_counts", y="pct_counts_mt", color="sample")

### Number of features (genes) per cell vs read counts before and after filter

In [ ]:
ax = sc.pl.scatter(adatas_concatraw, x="total_counts", y="n_genes", color="sample",show=False)
ax.axhline(y=scanpy_min_genes,color = "black")

In [ ]:
ax = sc.pl.scatter(adatas_concat, x="total_counts", y="n_genes", color="sample",show=False)

### Number of features (genes) per cell vs percentage mt genes before and after filter

In [ ]:
ax = sc.pl.scatter(adatas_concatraw, x="n_genes", y="pct_counts_mt", color="sample",show=False)
ax.axvline(x=scanpy_min_genes,color = "black")
ax.axhline(y=scanpy_pct_mt,color = "red")


In [ ]:
ax = sc.pl.scatter(adatas_concat, x="n_genes", y="pct_counts_mt", color="sample",show=False)

#### Voilin plot for number of genes before and after filter

In [ ]:
ax = sc.pl.violin(adatas_concatraw, ['n_genes'], groupby = 'sample',
             jitter=0.4, multi_panel=True, show=False)
ax.tick_params(axis='x', labelrotation=45)
ax.axhline(y=scanpy_min_genes)

In [ ]:
ax = sc.pl.violin(adatas_concat, ['n_genes'], groupby = 'sample',
             jitter=0.4, multi_panel=True, show=False)
ax.tick_params(axis='x', labelrotation=45)

#### Voilin plot for percentage of mt genes before and after filter

In [ ]:
ax = sc.pl.violin(adatas_concatraw, ['pct_counts_mt'], groupby = 'sample',
             jitter=0.4, multi_panel=True, show=False)
ax.tick_params(axis='x', labelrotation=45)
ax.axhline(y=scanpy_pct_mt)

In [ ]:
ax = sc.pl.violin(adatas_concat, ['pct_counts_mt'], groupby = 'sample',
             jitter=0.4, multi_panel=True,show=False)
ax.tick_params(axis='x', labelrotation=45)

#### Save h5ad files

In [ ]:
for i in range(len(adatas)):
  #adatas[i].write(str(SRR[i])+'.h5ad')
  adatas[i].write(os.path.splitext(os.path.basename(filelist[i]))[0]+'.h5ad')
